In [1]:
from edgar_functions_V2 import *
import numpy as np

In [2]:
tickers = [
    "META",
    "AMD",
    "WSM",
    "NVDA",
    "COST",
    "AAPL",
    "TSLA",
    "PLTR",
    "BAC",
    "F",
    "AAL",
    "SQ",
    "AMZN",
    "CCL",
    "SOFI",
    "INTC",
    "XOM",
    "RIVN",
    "MRO",
    "DAL",
    "JPM",
    "TM",
    "DE",
    "UPS",
    "PYPL",
]

In [3]:
scorecard_order = [
    "Net Revenue",
    "Cost of Revenue",
    "Gross Profit",
    "Gross Margin %",
    "SG&A",
    "Operating Income",
    "Operating Margin %",
    "EBT",
    "Tax Expense",
    "Tax Rate %",
    "Net Income",
    "EPS",
    "EPS Diluted",
    "Dividend",
    "Dividend Payout Ratio %",
    "Cash",
    "Marketable Securities",
    "Inventory",
    "Accounts Receivable",
    "Other Current Assets",
    "Total Current Assets",
    "Property, Plant, and Eqipment",
    "Other Long Term Assets",
    "Goodwill",
    "Total Assets",
    "Accounts Payable",
    "Accrued Expenses",
    "Deferred Revenue",
    "Income Tax Payable",
    "Total Current Liabilities",
    "Long Term Debt",
    "Other Long Term Liabilities",
    "Total Liabilities",
    "Additional Paid In Capital",
    "Retained Earnings",
    "Total Stockholders Equity",
    "Total Liabilities and Stockholders Equity",
    "Shares Outstanding (Diluted)",
    "Book Value Per Share",
    "Operating Cash Flow",
    "OCF/NI",
    "Depreciation and Amortization",
    "Capex",
    "Capex / Depreciation",
    "Free Cash Flow",
    "Dividend Payment",
    "Stock Repurchase",
    "ROA %",
    "ROE %",
    "Profit Margin %",
    "Asset Turnover",
    "Current Ratio",
    "Equity Multiplier",
    "Net Working Capital",
    "Debt to Equity Ratio",
    "Debt to Assets Ratio",
    "Days Sales Outstanding",
    "Days of Inventory on Hand",
    "Payables Period",
    "Receivables Turnover",
    "Cash Conversion Cycle",
]

In [4]:
ticker = tickers[4]
annual_df = process_and_aggregate_annual_data(ticker)
income_mapping = income_statement_mapping.income_mapping
balance_mapping = balance_sheet_mapping.balance_mapping
cash_mapping = cash_flow_mapping.cash_mapping
income = get_changed_columns(income_mapping, annual_df).droplevel(1, axis=1)
balance = get_changed_columns(balance_mapping, annual_df).droplevel(1, axis=1)
cash_flow = get_changed_columns(cash_mapping, annual_df).droplevel(1, axis=1)

income_df = income.T.groupby(income.columns).mean().sort_index(ascending=False)
income_df = income_df.sort_index(axis=1, ascending=False)
income_df = income_df.T
balance_df = balance.T.groupby(balance.columns).mean().sort_index(ascending=False)
balance_df = balance_df.T.sort_index(axis=1, ascending=False)
cash_df = cash_flow.T.groupby(cash_flow.columns).mean().sort_index(ascending=False)
cash_df = cash_df.T.sort_index(axis=1, ascending=False)

In [5]:
cash_df.T

,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Tax Withholding Related to Stock Based Compensation,NaN,NaN,"42,000,000","61,000,000","107,000,000","121,000,000","164,000,000","178,000,000","220,000,000","202,000,000","217,000,000","272,000,000","330,000,000","312,000,000","363,000,000","303,000,000"
Tax Benefit Related to Stock Based Compensation operating,"41,000,000","2,000,000","10,000,000","45,000,000","57,000,000","47,000,000","55,500,000","86,000,000","74,000,000","38,000,000",NaN,NaN,NaN,NaN,NaN,NaN
Tax Benefit Related to Stock Based Compensation financing,"41,000,000","2,000,000","10,000,000","45,000,000","64,000,000","61,000,000","84,000,000","86,000,000","74,000,000","38,000,000",NaN,NaN,NaN,NaN,NaN,NaN
Stock Based Compensation Expense,"166,000,000","181,000,000","190,000,000","207,000,000","241,000,000","285,000,000","327,000,000","394,000,000","459,000,000","514,000,000","544,000,000","595,000,000","619,000,000","665,000,000","724,000,000","774,000,000"
Restructuring Charges,NaN,"17,000,000","8,000,000","9,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Repurchases of Common Stock,"895,000,000","69,000,000","551,000,000","624,000,000","632,000,000","36,000,000","334,000,000","481,000,000","486,000,000","469,000,000","328,000,000","247,000,000","196,000,000","496,000,000","439,000,000","676,000,000"
Repayments of Short Term Debt,"5,163,000,000","1,777,000,000","73,000,000","105,000,000","114,000,000","287,000,000","103,000,000","51,000,000","106,000,000",0,0,NaN,NaN,0,"6,000,000","935,000,000"
Repayment of Long Term Debt,"69,000,000","6,000,000","84,000,000",0,"900,000,000",0,0,"1,000,000","1,288,000,000","2,200,000,000","86,000,000","89,000,000","3,200,000,000","94,000,000","800,000,000","75,000,000"
Realized Foreign Exchange Loss,NaN,NaN,"14,000,000","9,000,000","40,000,000","39,000,000","26,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Purchase of Property, Plant, and Equipment","1,599,000,000","1,250,000,000","1,055,000,000","1,290,000,000","1,480,000,000","2,083,000,000","1,993,000,000","2,393,000,000","2,649,000,000","2,502,000,000","2,969,000,000","2,998,000,000","2,810,000,000","3,588,000,000","3,891,000,000","4,323,000,000"


In [6]:
# for income statement
if "Cost of Revenue" not in income_df.columns:
    income_df["Cost of Revenue"] = income_df["Net Revenue"] - income_df["Gross Profit"]
if "Gross Profit" not in income_df.columns:
    income_df["Gross Profit"] = income_df["Net Revenue"] - income_df["Cost of Revenue"]
if "Tax Rate Continuing Operations" not in income_df.columns:
    income_df["Tax Rate Continuing Operations"] = -income_df["Taxes"] / income_df["EBT"]
if "SG&A" not in income_df.columns:
    income_df["SG&A"] = (
        income_df["Selling and Marketing"] + income_df["General and Administrative"]
    )
# for balance sheet
if "Total Liabilities" not in balance_df.columns:
    balance_df["Total Liabilities"] = (
        balance_df["Total Liabilities and Stockholders Equity"]
        - balance_df["Total Stockholders Equity"]
    )

if "Total Stockholders Equity" not in balance_df.columns:
    balance_df["Total Stockholders Equity"] = (
        balance_df["Total Assets"] - balance_df["Total Liabilities"]
    )
else:
    balance_df["Total Stockholders Equity"] = balance_df[
        "Total Stockholders Equity"
    ].fillna(balance_df["Total Assets"] - balance_df["Total Liabilities"])
# for cash flow
cash_df["Depreciation and Amortization(calc)"] = combine_or_add_columns(
    cash_df, ["Depreciation on Assets", "Amortization"], add_instead=True
)

In [7]:
scorecard = pd.DataFrame()
scorecard["Net Revenue"] = combine_or_add_columns(income_df, ["Net Revenue"])
scorecard["Cost of Revenue"] = combine_or_add_columns(income_df, ["Cost of Revenue"])
scorecard["Gross Profit"] = combine_or_add_columns(income_df, ["Gross Profit"])
scorecard["Gross Margin %"] = scorecard["Gross Profit"] / scorecard["Net Revenue"] * 100
scorecard["SG&A"] = combine_or_add_columns(income_df, ["SG&A"])
scorecard["Operating Income"] = combine_or_add_columns(income_df, ["Operating Income"])
scorecard["Operating Margin %"] = (
    scorecard["Operating Income"] / scorecard["Net Revenue"]
) * 100
scorecard["EBT"] = combine_or_add_columns(
    income_df, ["EBT", "EBT and equity investments"]
)
scorecard["Tax Expense"] = combine_or_add_columns(income_df, ["Taxes"])
scorecard["Tax Rate %"] = combine_or_add_columns(
    income_df, ["Tax Rate Continuing Operations"]
)
scorecard["Tax Rate %"] = scorecard["Tax Rate %"] * 100
scorecard["Net Income"] = combine_or_add_columns(income_df, ["Net Income"])
scorecard["EPS"] = combine_or_add_columns(income_df, ["EPS"])
scorecard["EPS Diluted"] = combine_or_add_columns(income_df, ["EPS Diluted"])
scorecard["Cash"] = combine_or_add_columns(
    balance_df, ["Cash", "Cash and Restricted Cash"]
)
scorecard["Marketable Securities"] = combine_or_add_columns(
    balance_df, ["Marketable Securities"]
)
scorecard["Inventory"] = combine_or_add_columns(balance_df, ["Inventory"])
scorecard["Accounts Receivable"] = balance_df["Accounts Receivable"]
scorecard["Other Current Assets"] = combine_or_add_columns(
    balance_df, ["Other Current Assets", "Prepaid Expenses and Other Current Assets"]
)
scorecard["Total Current Assets"] = balance_df["Total Current Assets"]
scorecard["Property, Plant, and Eqipment"] = balance_df[
    "Property, Plant, and Equipment"
]
scorecard["Other Long Term Assets"] = combine_or_add_columns(
    balance_df, ["Other Long Term Assets"]
)
scorecard["Goodwill"] = balance_df["Goodwill Asset"]
scorecard["Total Assets"] = balance_df["Total Assets"]
scorecard["Accounts Payable"] = combine_or_add_columns(balance_df, ["Accounts Payable"])
scorecard["Accrued Expenses"] = combine_or_add_columns(balance_df, ["Accrued Expenses"])
scorecard["Deferred Revenue"] = combine_or_add_columns(balance_df, ["Deferred Revenue"])
scorecard["Income Tax Payable"] = combine_or_add_columns(
    balance_df, ["Income Taxes Payable"]
)
scorecard["Operating Lease Liability"] = combine_or_add_columns(
    balance_df, ["Operating Lease Liability"]
)
scorecard["Total Current Liabilities"] = balance_df["Total Current Liabilities"]
scorecard["Long Term Debt"] = combine_or_add_columns(
    balance_df, ["Long Term Debt", "Long Term Lease Liability"], replace_zero=True
)
scorecard["Other Long Term Liabilities"] = combine_or_add_columns(
    balance_df, ["Other Long Term Liabilities"]
)
scorecard["Total Liabilities"] = combine_or_add_columns(
    balance_df, ["Total Liabilities"]
)
scorecard["Additional Paid In Capital"] = combine_or_add_columns(
    balance_df, ["Additional Paid in Capital"]
)
scorecard["Retained Earnings"] = combine_or_add_columns(
    balance_df, ["Retained Earnings"]
)
scorecard["Total Stockholders Equity"] = combine_or_add_columns(
    balance_df, ["Total Stockholders Equity", "Total Equity(calc)"]
)
scorecard["Total Liabilities and Stockholders Equity"] = combine_or_add_columns(
    balance_df,
    ["Total Liabilities and Stockholders Equity", "Toal Liabilites and Equity(calc)"],
)
scorecard["Shares Outstanding (Diluted)"] = combine_or_add_columns(
    income_df,
    ["Weighted Average Shares Diluted", "Weighted Average Shares Outstanding"],
    replace_zero=True,
)
scorecard["Book Value Per Share"] = (
    scorecard["Total Stockholders Equity"] / scorecard["Shares Outstanding (Diluted)"]
)
scorecard["Operating Cash Flow"] = combine_or_add_columns(
    cash_df, ["Net Cash Provided by Operating Activities"]
)
scorecard["OCF/NI"] = scorecard["Operating Cash Flow"] / scorecard["Net Income"]
scorecard["Depreciation and Amortization"] = combine_or_add_columns(
    cash_df, ["Depreciation and Amortization", "Depreciation and Amortization(calc)"]
)
scorecard["Capex"] = combine_or_add_columns(
    cash_df, ["Purchase of Property, Plant, and Equipment"]
)
scorecard["Capex"] = -scorecard["Capex"]
scorecard["Capex / Depreciation"] = (
    scorecard["Capex"] / scorecard["Depreciation and Amortization"]
)
scorecard["Free Cash Flow"] = scorecard["Operating Cash Flow"] + scorecard["Capex"]
scorecard["Dividend Payment"] = combine_or_add_columns(
    cash_df, ["Payment of Dividends"]
)
if scorecard["Dividend Payment"].isna().all():
    pass
else:
    scorecard["Dividend Payment"] = -scorecard["Dividend Payment"]
scorecard["Stock Repurchase"] = combine_or_add_columns(
    cash_df, ["Repurchases of Common Stock"]
)
scorecard["Stock Repurchase"] = -scorecard["Stock Repurchase"]
scorecard["ROA %"] = (scorecard["Net Income"] / scorecard["Total Assets"]) * 100
scorecard["ROE %"] = (
    scorecard["Net Income"] / scorecard["Total Stockholders Equity"]
) * 100
scorecard["Profit Margin %"] = scorecard["Net Income"] / scorecard["Net Revenue"] * 100
scorecard["Asset Turnover"] = scorecard["Net Revenue"] / scorecard["Total Assets"]
scorecard["Current Ratio"] = (
    scorecard["Total Current Assets"] / scorecard["Total Current Liabilities"]
)
scorecard["Equity Multiplier"] = (
    scorecard["Total Assets"] / scorecard["Total Stockholders Equity"]
)
scorecard["Net Working Capital"] = (
    scorecard["Total Current Assets"] - scorecard["Total Current Liabilities"]
)
scorecard["Debt to Equity Ratio"] = (
    scorecard["Total Liabilities"] / scorecard["Total Stockholders Equity"]
)
scorecard["Debt to Assets Ratio"] = (
    scorecard["Total Liabilities"] / scorecard["Total Assets"]
)
scorecard["Days Sales Outstanding"] = (
    scorecard["Accounts Receivable"] / scorecard["Net Revenue"]
) * 365
scorecard["Days of Inventory on Hand"] = (
    scorecard["Inventory"] / scorecard["Cost of Revenue"]
) * 365
scorecard["Payables Period"] = (
    scorecard["Accounts Payable"] / scorecard["Cost of Revenue"]
) * 365
scorecard["Receivables Turnover"] = (
    scorecard["Net Revenue"] / scorecard["Accounts Receivable"]
)
scorecard["Cash Conversion Cycle"] = (
    scorecard["Days Sales Outstanding"]
    + scorecard["Days of Inventory on Hand"]
    - scorecard["Payables Period"]
)
if scorecard["Dividend Payment"].isna().all():
    scorecard["Dividend"] = np.nan
else:
    scorecard["Dividend"] = (
        -scorecard["Dividend Payment"] / scorecard["Shares Outstanding (Diluted)"]
    )
if "Dividend" in scorecard.columns:
    scorecard["Dividend Payout Ratio %"] = (
        scorecard["Dividend"] / scorecard["EPS"]
    ) * 100
else:
    scorecard["Dividend Payout Ratio %"] = np.nan
scorecard = scorecard[scorecard_order]

In [8]:
scorecard.T

,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008
Net Revenue,"242,290,000,000","226,954,000,000","195,929,000,000","166,761,000,000","152,703,000,000","141,576,000,000","128,074,000,000","117,837,000,000","114,932,500,000","111,426,000,000","104,013,000,000","98,099,500,000","87,981,500,000","77,100,500,000","70,655,500,000","71,730,000,000"
Cost of Revenue,"212,586,000,000","199,382,000,000","170,684,000,000","144,939,000,000","132,886,000,000","123,152,000,000","111,882,000,000","102,901,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gross Profit,NaN,NaN,NaN,NaN,"16,465,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gross Margin %,NaN,NaN,NaN,NaN,10.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SG&A,"21,590,000,000","19,779,000,000","18,537,000,000","16,387,000,000","14,994,000,000","13,876,000,000","12,950,000,000","12,068,000,000","11,445,000,000","10,899,000,000","10,104,000,000","9,518,000,000","8,691,000,000","7,848,000,000","7,252,000,000","6,954,000,000"
Operating Income,"8,114,000,000","7,793,000,000","6,708,000,000","5,435,000,000","4,737,000,000","4,480,000,000","4,111,000,000","3,672,000,000","3,624,000,000","3,220,000,000","3,053,000,000","2,759,000,000","2,439,000,000","2,077,000,000","1,777,000,000","1,969,000,000"
Operating Margin %,3.35,3.43,3.42,3.26,3.10,3.16,3.21,3.12,3.15,2.89,2.94,2.81,2.77,2.69,2.52,2.75
EBT,"8,487,000,000","7,840,000,000","6,680,000,000","5,367,000,000","4,765,000,000","4,442,000,000","4,039,000,000","3,619,000,000","3,604,000,000","3,197,000,000","3,051,000,000","2,767,000,000","2,383,000,000","2,054,000,000","1,727,000,000",NaN
Tax Expense,"2,195,000,000","1,925,000,000","1,601,000,000","1,308,000,000","1,061,000,000","1,263,000,000","1,325,000,000","1,243,000,000","1,195,000,000","1,109,000,000","990,000,000","1,000,000,000","841,000,000","731,000,000","628,000,000","716,000,000"
Tax Rate %,25.90,24.60,24,24.40,22.30,28.40,32.80,34.30,33.20,34.70,32.40,36.10,35.30,35.60,36.40,35.60
